# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error --> same as margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [56]:
import pandas as pd
import numpy as np
from scipy import stats

In [57]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [58]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [59]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


### Q1
What test is appropriate for this problem? Does CLT apply?

This is a case of two independant sample test.  
A permuatation based bootstrapping test is most appropriate for this problem.
We can decide on a test statistic and can calculate the p-value from the permuted botstraped samples. 
The p-value shall help us to decide whether the samples are identically distributed or not. 

Central Limit Theorem(CLT) can be applied to any dataset irrespective of the underlying distribution. 
So yes CLT can be applied. 

### Q2. 
What are the null and alternate hypotheses?

#### Null Hypothesis: 
    Race has no significance on the rate of callbacks for the resumes.
    
#### Alternate Hypothesis:
    Race has significance on the rate of callbacks for the resumes.
    
#### Test Statistic: 
    Fraction of callbacks to the black community to the total number of resumes received. 
    

### Q3 
Compute margin of error --> same as margin of error, confidence interval, and p-value. 
Try using both the bootstrapping and the frequentist statistical approaches.

In [60]:
w = data[data.race=='w']
b = data[data.race=='b']


#### Bootstrapping Approach: 

##### Hypothesis Testing:

In [70]:
def fract_b_yea(w, b):
    frac = np.sum(b) / (len(w) + len(b))
    return frac

def draw_perm_reps(x, y, func, size= 1):
    total = pd.concat([x,y])
    perm_reps_hypo = np.empty(size)
    perm_reps      = np.empty(size)
    
    for i in range(size):
        total_perm = np.random.permutation(total)
        x_perm = total_perm[:len(x)]
        y_perm = total_perm[len(x):]
        perm_reps_hypo[i] = func(x_perm, y_perm)
        perm_reps[i]      = x_perm.mean() - y_perm.mean()
        
    return perm_reps_hypo, perm_reps


perm_reps_hypo, perm_reps = draw_perm_reps(w['call'], b['call'], fract_b_yea, 10000)
p_value = np.sum(perm_reps_hypo <= np.sum(b['call'])/(len(w) + len(b)))
#print(np.sum(b['call'])/(len(w['call'])+len(b['call'])))
print(f'p_value: {p_value}')
if p_value < 0.05:
    print("We reject the Null at 95% significance level.\nHence the black and white samples are not identical and Race has significance on the rate of callbacks")
else:
    print("We accept the Null at 95% significance level.\nHence the black and white samples are identical and Race has no significance on the rate of callbacks")

p_value: 0
We reject the Null at 95% significance level.
Hence the black and white samples are not identical and Race has significance on the rate of callbacks


##### Confidence Interval:

In [71]:
ci = np.percentile(perm_reps, [2.5,97.5])
print(f'Confidence Interval: {ci}')

Confidence Interval: [-0.01560576  0.01560576]


##### Margin of Error

In [72]:
me = (ci[1] - ci[0])/2
print(f'Margin of Error: {me}')

Margin of Error: 0.015605755150318146


#### Frequentist Approach

##### Hypothesis Testing:

###### This is a case of two independant sample test with same sample size. 
Standard Error = $\sqrt{\frac{S1^2}{n1} + \frac{S2^2}{n2}}\$

###### Null Hypothesis:
Race has no significance on the rate of callbacks for the resumes.

###### Considerations: 
- Check if both the samples of black as well as white are identically distributed. 
  Hence the difference in their means is zero
- We check for 95% significance level. Hence $\alpha$ = 0.05
- We condider it be a two tailed test. 

In [73]:
s1 = np.std(w['call'])
s2 = np.std(b['call'])
n  = len(w['call'])
x1 = np.mean(w['call'])
x2 = np.mean(b['call'])


se = np.sqrt(s1**2/n + s2**2/n)

z = (x1 - x2 - 0)/se 
z_critical = 1.96

print(z)

if z > z_critical or z < -z_critical:
    print("We reject the Null at 95% significance level.\nHence the black and white samples are not identical and Race has significance on the rate of callbacks")
else:
    print("We accept the Null at 95% significance level.\nHence the black and white samples are identical and Race has no significance on the rate of callbacks")

4.115583422082968
We reject the Null at 95% significance level.
Hence the black and white samples are not identical and Race has significance on the rate of callbacks


##### Calculating the Confidence interval:
${\mu \pm z_* * Standard Error}$ 

- $\mu$ ---> $\mu_1 - \mu_2$
- For 95% confidence level ---> $z_* = 1.96$

In [65]:
ci_upper = (x1 - x2) + 1.96*se
ci_lower = (x1 - x2) - 1.96*se
print(f'Confidence Interval - [{ci_lower}, {ci_upper}]')

Confidence Interval - [0.016777570469610682, 0.04728813924051047]


##### Margin of Error

In [66]:
me = (ci_upper - ci_lower)/2
print(f'Margin of Error: {me}')

Margin of Error: 0.015255284385449895


### Q4
Write a story describing the statistical significance in the context or the original problem.
 

By bootatrapping technique, we get the p_value as 0.0.
This suggests that there is 0% likelihood that the fraction of black calls are by chance. 
Hence we can reject the null and accept the alternate hypothesis. 

By frequentist technique, the z-score calculated is 4.16. 
This is higher than 1.96 which is our z_critical value for a two tailed test. 
The z-score > z_critical indicates that difference of mean lies way ahead of 1.96*standard error. 
Hence the distributions are not identical and we can conculde that alternate hypothesis is true. 

### Q5
Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Yes Race/Name is a very important factor for callback success as per the data provided and the analysis done. 
The sample has 2435 resumes marked as black names and 2435 resumes marked as white names.  
Off 4870 candidates only 392 received a callback.
Almost 60% of white candidates received a callback whereas only 40% of black candidates received a callback. 

In [69]:
## Calculations for above: 
print(np.sum(b['call']) / (np.sum(w['call']) + np.sum(b['call'])))
print(np.sum(w['call']) / (np.sum(w['call']) + np.sum(b['call'])))
print(np.sum(w['call']) + np.sum(b['call']), np.sum(w['call']), np.sum(b['call']))
print(len(w['call']), len(b['call']))

0.4005102
0.5994898
392.0 235.0 157.0
2435 2435
